In [ ]:
import requests, bs4, re
import pandas as pd
from datetime import date, timedelta

myDate = date(2012, 1, 3)
today = date.today()
plusOne = timedelta(days=1)
datePattern = re.compile('\d{4}\-\D{3}\-\d{2}')
url_format = 'http://www.nasdaq.com/earnings/earnings-calendar.aspx?date=%s'
dateList = []
tickerList = []

# Request Nasdaq.com Earnings Page for examination
while today > myDate:
    try:
        res = requests.get(url_format % myDate.strftime("%Y-%b-%d"))
    except:
        myDate += plusOne
        continue
    else:
        if myDate.weekday() not in [5,6]:
            res.raise_for_status()
            NasSoup = bs4.BeautifulSoup(res.text,"html5lib")
            
            try:
                #Save companies reporting earnings to a list
                companies = NasSoup.find_all(href=re.compile("/earnings/report"), id=True)
            except:
                myDate += plusOne
                continue
            
            # Search pattern to discover ticker symbols
            tickPattern = re.compile(r'\([A-Z]+\)')
            # Extract Tickers
            for ticker in companies:
                try:
                    tickerList.append(re.findall(tickPattern,str(ticker))[-1].strip('()'))
                    dateList.append(myDate)
                except IndexError:
                    pass
    finally:
        myDate += plusOne

data = {'Equities': tickerList, 'ReportDate': dateList}
df = pd.DataFrame(data)

In [ ]:
url_formatted = 'http://www.marketwatch.com/investing/stock/%s/financials'
eqList = df.Equities.unique()
eEPS = {}

# Navigate to website for each unique ticker
for ticker in eqList:
    try:
        res = requests.get(url_formatted % ticker)
    except:
        EPS = "N/A"
        continue
    else:
        res.raise_for_status()
        epsSoup = bs4.BeautifulSoup(res.text,"html5lib")
        
        try:
            #Find the closest link with id value and go to previous sibling for earnings data
            closeLink = epsSoup.find('tr', id="ratio_Eps1YrAnnualGrowth")
            EPS = closeLink.find_previous_sibling("tr")
        except:
            EPS = "N/A"
            continue
            
        # Extract 2012-2016 EPS data and save to list EarnList 4 times for each quarter's earnings
        Earnings = EPS.find_all('td')
        EarnList=[]
        for ele in Earnings:
            if ele.get_text() in [' EPS (Basic)','']:
                pass
            else:
                for i in range(4):
                    EarnList.append(ele.get_text())
        eEPS[ticker] = EarnList